#**데이터 불러오기**

In [2]:
#사용자 u.user 파일을 DataFrame으로 읽기 
import os 
import pandas as pd 

base_src = 'drive/MyDrive/RecoSys/Data'
u_user_src = os.path.join(base_src,'u.user')
u_cols = ["user_id","age",'sex',"occupation","zip_code"]
users = pd.read_csv(u_user_src,
                    sep = '|',
                    names = u_cols,
                    encoding='latin-1')
users = users.set_index('user_id')
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [28]:
#사용자 u.item 파일을 DataFrame으로 읽기 
u_item_src = os.path.join(base_src,'u.item')
i_cols = ["movie_id","title","release date","video release date","IMDB URL","unknown","Action","Adventure","Animaiton",
          "Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
          "Film-Noir","Horror", "Musical",'Mystery','Romance','Sci-Fi',"Thriller","War",'Western']
movies = pd.read_csv(u_item_src,
                    sep = '|',
                    names = i_cols,
                    encoding='latin-1')
movies = movies.set_index('movie_id')
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animaiton,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [29]:
#사용자 u.data 파일을 DataFrame으로 읽기 
u_data_src = os.path.join(base_src,'u.data')
r_cols = ["user_id","movie_id",'rating','timestamp']
ratings = pd.read_csv(u_data_src,
                    sep = '\t',
                    names = r_cols,
                    encoding='latin-1')
ratings = ratings.set_index('user_id')
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


#**2.2 인기제품 방식**

In [23]:
#인기 제품 방식 추천 function 
def recom_movie(n_items):
  movie_mean = ratings.groupby(['movie_id'])['rating'].mean() #movie_id를 기준으로 groupby 후 점수를 평균
  movie_sort = movie_mean.sort_values(ascending=False)[:n_items] #영화 평균을 정렬하고 몇 개를 뽑아낼 건지 
  recom_movies = movies.loc[movie_sort.index] #뽑힌 무비 솔트의 인덱스를 기준으로 movies data frame에서 조회 
  recommendations = recom_movies["title"]
  return recommendations

recom_movie(5)

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

#**2.3 추천 시스템의 정확도 측정**

In [25]:
#100K의 영화 평점에 대해서 실제값과 best-seller 방식으로 구한 예측값의 RMSE를 계산하는 코드 
import numpy as np 

def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

#정확도 계산 
rmse = []
movie_mean = ratings.groupby(['movie_id'])['rating'].mean() #각 영화의 평점 계산

for user in set(ratings.index):
  y_true = ratings.loc[user]['rating'] #각 사용자가 평가한 모든 영화의 평점 
  #best-seller 방식으로 
  y_pred = movie_mean[ratings.loc[user]['movie_id']]
  accuracy = RMSE(y_true, y_pred)
  rmse.append(accuracy)

#RMSE 계산
print(np.mean(rmse)) #0.996007224010567

0.996007224010567


#**2.4 사용자 집단별 추천**

In [36]:
#ratings DataFrame에서 timestamp 제거
base_src = 'drive/MyDrive/RecoSys/Data'
u_user_src = os.path.join(base_src,'u.user')
u_cols = ["user_id","age",'sex',"occupation","zip_code"]
users = pd.read_csv(u_user_src,
                    sep = '|',
                    names = u_cols,
                    encoding='latin-1')

u_data_src = os.path.join(base_src,'u.data')
r_cols = ["user_id","movie_id",'rating','timestamp']
ratings = pd.read_csv(u_data_src,
                    sep = '\t',
                    names = r_cols,
                    encoding='latin-1') 
ratings = ratings.drop('timestamp', axis=1)
u_item_src = os.path.join(base_src,'u.item')

i_cols = ["movie_id","title","release date","video release date","IMDB URL","unknown","Action","Adventure","Animaiton",
          "Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
          "Film-Noir","Horror", "Musical",'Mystery','Romance','Sci-Fi',"Thriller","War",'Western']
movies = pd.read_csv(u_item_src,
                    sep = '|',
                    names = i_cols,
                    encoding='latin-1')
movies = movies[['movie_id','title']]

In [37]:
#데이터 train, test set 분리
from sklearn.model_selection import train_test_split 

x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,stratify=y) #stratify : 계층화 추출 

#정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

#모델별 RMSE를 계산하는 함수 
def score(model): #내가 만든 모델의 성능 
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user,movie) for (user,movie) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

#best-seller 함수를 이용한 정확도 계산
train_mean = x_train.groupby(['movie_id'])['rating'].mean()
def best_seller(user_id, movie_id):
  try:
    rating = train_mean[movie_id] 
  except: 
    rating = 3.0 #train에 없지만 test에 있는 경우 
  return rating

score(best_seller) #1.0300575720087572

1.0311048674612568

In [38]:
#성별에 따른 예측값 계산 
merged_ratings = pd.merge(x_train, users) #user_id를 기준으로 합치기 
users = users.set_index('user_id')

g_mean = merged_ratings[['movie_id','sex','rating']].groupby(['movie_id','sex'])['rating'].mean() #여자의 영화 평균


movie_id  sex
1         F      3.782178
          M      3.913580
2         F      3.500000
          M      3.185185
3         F      3.000000
                   ...   
1677      F      3.000000
1678      M      1.000000
1679      M      3.000000
1680      M      2.000000
1681      M      3.000000
Name: rating, Length: 3032, dtype: float64

In [41]:
rating_matrix = x_train.pivot(index='user_id',
                              columns ='movie_id',
                              values = 'rating')
rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1638,1639,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1652,1653,1654,1655,1656,1657,1658,1659,1660,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1677,1678,1679,1680,1681
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,NaN,3.0,3.0,5.0,NaN,1.0,NaN,NaN,2.0,5.0,NaN,NaN,5.0,5.0,NaN,4.0,5.0,4.0,NaN,NaN,4.0,3.0,4.0,3.0,NaN,4.0,1.0,3.0,3.0,5.0,4.0,NaN,1.0,2.0,2.0,3.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,3.0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
#Gender 기준 추천 
def cf_gender(user_id, movie_id):
  if movie_id in rating_matrix.columns: #영화가 pivot table에 있는지 없는지 
    gender = users.loc[user_id]['sex']
    if gender in g_mean[movie_id].index: #성별이 영화 평가에 있는지 없는지
      gender_rating = g_mean[movie_id][gender]
    else: 
      gender_rating = 3.0
  else: 
    gender_rating = 3.0
  return gender_rating

score(cf_gender) #1.038721110980759

1.038721110980759